# Data Download using Azure FarmBeats
Download the required satellite and weather data using Azure FarmBeats PaaS.

### Import 3rd party libraies

In [ ]:
# Disable unnecessary logs 
import sys
import logging
logging.disable(sys.maxsize)

from datetime import datetime
import json
import numpy as np
import os
import pandas as pd
import rasterio
from azure.identity import ClientSecretCredential

### Import Farmbeats and Utilities

In [ ]:
from azure.farmbeats.models import Farmer, Boundary, Polygon, SatelliteIngestionJobRequest, WeatherIngestionJobRequest
from azure.farmbeats import FarmBeatsClient

from utils.config import farmbeats_config
from utils.weather_util import WeatherUtil
from utils.satellite_util import SatelliteUtil
from utils.constants import CONSTANTS

### Farmbeats Configuration

In [ ]:
# FarmBeats Client definition
credential = ClientSecretCredential(
    tenant_id=farmbeats_config['tenant_id'],
    client_id=farmbeats_config['client_id'],
    client_secret=farmbeats_config['client_secret'],
    authority=farmbeats_config['authority']
)

credential_scopes = [farmbeats_config['default_scope']]

fb_client = FarmBeatsClient(
    base_url=farmbeats_config['instance_url'],
    credential=credential,
    credential_scopes=credential_scopes,
    logging_enable=True
)

In [ ]:
RUN = 56
NO_BOUNDARIES = 1
root_dir = "C:\\farmbeats\\" # Satellite data gets downloaded here

### Create Farmer

In [ ]:
farmer_id = "annam_farmer"
try:
    farmer = fb_client.farmers.get(farmer_id=farmer_id)
    if farmer is not None:
        print("Farmer Exists")
    else:
        print("Farmer doesn't exist...Creating one ", end="", flush=True)
        farmer = fb_client.farmers.create_or_update(
            farmer_id=farmer_id,
            farmer=Farmer()
        )
except Exception as e:
    print(e)

### Create Boundaries

In [ ]:
# Read 1000 farm geojsons from farms_1kmx1km.csv
locations_df = pd.read_csv("farms_sample_1kmx1km.csv")
locations_df["farms1"] = locations_df.farms.apply(json.loads)  # farm geojsons converted from string to list with numeric elements

boundaries = []

for i, item in enumerate(locations_df.farms1.values[:NO_BOUNDARIES]):
    boundary_id="boundary" + str(i) + str(RUN)
    try:
        boundary = fb_client.boundaries.get(
            farmer_id=farmer_id,
            boundary_id=boundary_id
        )
        if boundary is not None:
            print("Exist")
        else:
            print(f"Creating boundary with id {boundary_id}... ", end="")
            boundary = fb_client.boundaries.create_or_update(
                farmer_id=farmer_id,
                boundary_id=boundary_id,
                boundary=Boundary(
                    description="Created by SDK",
                    geometry=Polygon(
                        coordinates=[
                        item
                        ]
                    )
                )
            )
            print("Created")
    except Exception as e:
        print(e)
    boundaries.append(boundary)

###  Submit Satellite Jobs

In [ ]:
# Start and End data for Satellite and Weather data to be pulled
start_dt = datetime.strptime(CONSTANTS["interp_date_start"], "%d-%m-%Y")
end_dt = datetime.strptime(CONSTANTS["interp_date_end"], "%d-%m-%Y")

In [ ]:
satellite_jobs = []
for i, boundary in enumerate(boundaries[:NO_BOUNDARIES]):
    job_id = "satellitejob"+ str(i) + str(RUN)
    
    # Submit Satellite Job
    try:
        print("Queuing satellite job... ", end="", flush=True)
        satellite_job = fb_client.scenes.begin_create_satellite_data_ingestion_job(
            job_id=job_id,
            job=SatelliteIngestionJobRequest(
                farmer_id=boundary.farmer_id,
                boundary_id=boundary.id,
                start_date_time=start_dt,
                end_date_time=end_dt,
            ),
            polling=True
        )
        print("Submitted Satellite Job")

    except HttpResponseError as e:
        print(e)
        raise
    satellite_jobs.append(satellite_job)

### Check status of Satellite Jobs

In [ ]:
for sat_job in satellite_jobs:
    print("Waiting")
    sat_job.result()

for sat_job in satellite_jobs:
    print(sat_job.result().as_dict()['id'])
    print(sat_job.status())
    
# TODO: Save job ids with Job request body to track failed jobs if any!

### Submit Weather (Historical) Jobs

In [ ]:
# Weather API inputs
extension_id = "dtn.clearAg"
extension_api_name = "dailyhistorical"
extension_data_provider_api_key = farmbeats_config["weather_provider_key"]
extension_data_provider_app_id = farmbeats_config["weather_provider_id"]

In [ ]:
weather_jobs = []
for i, boundary in enumerate(boundaries[:NO_BOUNDARIES]):
    job_id = "weatherjob"+ str(i) + str(RUN)
    st_unix = int(start_dt.timestamp())
    ed_unix = int(end_dt.timestamp())
    
    try:
        print("Queuing weather job... ", end="", flush=True)
        weather_job = fb_client.weather.begin_create_data_ingestion_job(
            job_id=job_id,
            job=WeatherIngestionJobRequest(
                farmer_id=boundary.farmer_id,
                boundary_id=boundary.id,
                extension_id= extension_id, 
                extension_api_name= extension_api_name, 
                extension_api_input= {"start": st_unix, "end": ed_unix},
                extension_data_provider_api_key= extension_data_provider_api_key,
                extension_data_provider_app_id=extension_data_provider_app_id
            ),
            polling=True
        )
        print("Submitted Weather Job")
    except HttpResponseError as e:
        print(e)
        raise
    weather_jobs.append(weather_job)

### Check status of Weather (Historical) Jobs

In [ ]:
for wth_job in weather_jobs:
    print("Waiting")
    wth_job.result()

for wth_job in weather_jobs:
    print(wth_job.result().as_dict()['id'])
    print(wth_job.status())
    
# TODO: Save job ids with Job request body to track failed jobs if any!

### Submit Weather (forecast) jobs

In [ ]:
from datetime import datetime, timedelta
weather_forecast_jobs = []
extension_api_name = "dailyforecast"
for i, boundary in enumerate(boundaries[:NO_BOUNDARIES]):
    job_id = "weatherforecastjob"+ str(i) + str(RUN)
    
    try:
        print("Queuing weather job... ", end="", flush=True)
        weather_job = fb_client.weather.begin_create_data_ingestion_job(
            job_id=job_id,
            job=WeatherIngestionJobRequest(
                farmer_id=boundary.farmer_id,
                boundary_id=boundary.id,
                extension_id= extension_id, 
                extension_api_name= extension_api_name, 
                extension_api_input= {"start": 0, "end": 10},
                extension_data_provider_api_key= extension_data_provider_api_key,
                extension_data_provider_app_id=extension_data_provider_app_id
            ),
            polling=True
        )
        print("Submitted Weather Job")
    except HttpResponseError as e:
        print(e)
        raise
    weather_forecast_jobs.append(weather_job)

In [ ]:
for wth_job in weather_forecast_jobs:
    print("Waiting")
    wth_job.result()

for wth_job in weather_forecast_jobs:
    print(wth_job.result().as_dict()['id'])
    print(wth_job.status())
    
# TODO: Save job ids with Job request body to track failed jobs if any!

### Download Satellite Data to Local

In [ ]:
df = SatelliteUtil(farmbeats_client = fb_client).download_and_get_sat_file_paths(farmer_id, boundaries, 
                                                                              start_dt, 
                                                                              end_dt, 
                                                                              root_dir)
df.to_csv("satellite_paths.csv", index=None)

### Download Weather Data to Local

In [ ]:
for boundary in boundaries:
    weather_list = fb_client.weather.list(
            farmer_id=  boundary.farmer_id,
            boundary_id= boundary.id,
            extension_id="dtn.clearAg", 
            weather_data_type= "historical", 
            granularity="daily")

    weather_data = []
    for w_data in weather_list:
        weather_data.append(w_data)

    w_df = WeatherUtil.get_weather_data_df(weather_data)
    w_df.to_csv(boundary.id + "_historical.csv", index=False)

In [ ]:
for boundary in boundaries:
    weather_list = fb_client.weather.list(
            farmer_id=  boundary.farmer_id,
            boundary_id= boundary.id,
            extension_id="dtn.clearAg", 
            weather_data_type= "forecast", 
            granularity="daily")

    weather_data = []
    for w_data in weather_list:
        weather_data.append(w_data)

    w_df = WeatherUtil.get_weather_data_df(weather_data)
    w_df.to_csv(boundary.id + "_forecast.csv", index=False)